In [1]:
import pandas as pd
import datetime
from scipy.stats import ttest_ind
from scipy import stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_web_logs = pd.read_csv('2022-04-13T12_df_web_logs.csv')
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
df_web_logs.head(3)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7


In [5]:
print(df_web_logs.shape)
print('The describe is\n\n',df_web_logs.describe())
print('The count is\n\n',df_web_logs['page'].value_counts())
print('Duplicates count is \n\n', df_web_logs[['user_id','page','date','load_time']].duplicated().sum())
print('Is NaN data', df_web_logs['load_time'].isna().sum())

(2401709, 4)
The describe is

           load_time
count  2.401709e+06
mean   7.397173e+01
std    1.104418e+02
min    0.000000e+00
25%    6.190000e+01
50%    7.000000e+01
75%    7.810000e+01
max    4.197500e+03
The count is

 m    1563381
b     588566
p     249762
Name: page, dtype: int64
Duplicates count is 

 15
Is NaN data 0


In [6]:
user_experiment = pd.read_csv('experiment_users.csv')
user_experiment.head(3)

,user_id,pilot
0,c36b2e,0
1,20336e,0
2,034652,0


In [11]:
print('Количество дублей',user_experiment.duplicated().sum(),'\n')
print('Распределение по группам\n',user_experiment['pilot'].value_counts())

Количество дублей 0 

Распределение по группам
 1    2773
0    2683
Name: pilot, dtype: int64


In [48]:
dt_beg = pd.datetime(2022,4,5)
dt_end = pd.datetime(2022,4,12)

df_metrics = (
    df_web_logs[(df_web_logs['date']>=dt_beg) & (df_web_logs['date']<dt_end)]
    .groupby('user_id')[['load_time']].agg(list)
    .reset_index()
)

df = pd.merge(user_experiment,df_metrics,on='user_id',how='left')
df.head()

,user_id,pilot,load_time
0,c36b2e,0,"[106.6, 49.6, 49.9, 75.7]"
1,20336e,0,"[61.6, 82.2, 67.4]"
2,034652,0,"[42.6, 68.6, 68.4, 63.5, 88.4, 68.9]"
3,e98e3b,0,"[62.3, 73.0, 80.5, 79.0, 73.9, 55.0, 64.0, 83...."
4,3f1105,0,"[84.0, 71.6, 64.2]"


In [49]:
def check_linearization(a, b):
    """Проверка гипотезы с помощью линеаризации.
    
    a: List[np.array], список множеств длин сессий пользователей контрольной группы
    b: List[np.array], список множеств длин сессий пользователей пилотной группы
    
    return: pvalue и точечную оценку.
    """
    a_x = np.array([np.sum(row) for row in a])
    a_y = np.array([len(row) for row in a])
    b_x = np.array([np.sum(row) for row in b])
    b_y = np.array([len(row) for row in b])
    coef = np.sum(a_x) / np.sum(a_y)
    a_lin = a_x - coef * a_y
    b_lin = b_x - coef * b_y
    _, pvalue = stats.ttest_ind(a_lin, b_lin)
    delta = np.mean(b_lin) - np.mean(a_lin)
    return pvalue, delta

In [51]:
pvalue = check_linearization(df[df['pilot']==0]['load_time'].values,df[df['pilot']==1]['load_time'].values)[0]

print(f'pvalue = {pvalue:0.4f}')

pvalue = 0.0442
